In [ ]:
import pandas as pd
import zipfile
import numpy as np
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
import pandas as pd
import folium
import json
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
zf = zipfile.ZipFile('/content/drive/MyDrive/House_Sale/House_Sale.zip') # alocando o arquivo zip
data = pd.read_csv(zf.open('kc_house_data.csv')) # abrindo o arquivo CSV 'train.csv' presente dentro do ZIP

In [ ]:
# verificando se há campos com valores NaN, como no treinamento da rede neural
# foi feito isso aqui porque no mapa serão usados todos os atributos da casa então é bom formatar o dataframe da maneira adequada
data.isnull().sum()

id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       2
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

In [ ]:
# descobrindo a localização exata dos valores NaN para que sejam removidos
np.where(data['sqft_above'].isnull().values==True)

(array([10, 17]),)

In [ ]:
data = data.drop([10, 17]) # retirando os valores NaN

In [ ]:

df=pd.DataFrame(data) 

# no outro notebook colab usei np.save para salvar o objeto np.array "new_price", e agora estou carregando tal objeto
new_price=np.load('new_price.npy') 

df['new_price']=new_price #adicionei ele no dataframe como uma nova coluna chamada new_price


In [ ]:
geo_json_data = json.load(open('/content/washington.json')) #carregando o arquivo .json do estado de Washington

In [ ]:
data=data.loc[:,:] # usando a função .loc() para acessar o dataframe inteiro

# criando um mapa e iniciando ele na localização de uma casa qualquer do dataframe, com zoom inicial de 8
mapa = folium.Map(location=[47.5112,-122.257], zoom_start=8) 

# sobrepondo o arquivo .json de washington no nosso mapa, destacando o estado e suas fronteiras
folium.GeoJson(geo_json_data).add_to(mapa)

In [ ]:
# uma alternativa pensada para reduzir a lentidão ao colocar todas as casas no mapa foi a clusterização dos marcadores.
# Essa técnica agrupa marcadores próximos, sem mostrar todos os marcadores de uma só vez
  # a técnica também foi usada na intenção de deixar o mapa mais bonito, limpo e com leitura fácil
  # pois ficaria feio mais de 21600 marcadores juntos e amontoados

marklus=MarkerCluster().add_to(mapa) # criando um objeto MarkerCluster() chamado marklus e adicionando ele ao nosso mapa

# o método iterrows() itera sobre cada linha do dataframe
for index, row in data.iterrows(): 

  # a variável popup_text armazena todos os dados que serão exibidos nos popups dos marcadores
  # aqui foi usado HTML e CSS para personalizar o estilo e aparência tanto do popup como do texto dentro dele
    popup_text = folium.Popup(f"<div style='font-size: 10pt;'><i><span style='font-family: Palatino, URW Palladio L, serif;'><b>ID:</b> {row['id']}.<br> <b>Data de venda:</b> {row['date']}.<br> <b>Preço:</b> {row['price']}.<br> <b>Quartos:</b> {row['bedrooms']}.<br> <b>Banheiros:</b> {row['bathrooms']}.<br> <b>Metragem quadrada da casa:</b> {row['sqft_living']}.<br> <b>Metragem quadrada do lote:</b> {row['sqft_lot']}.<br> <b>Andares:</b> {row['floors']}.<br> <b>Casa beira-mar:</b> {row['waterfront']}.<br> <b>Visita:</b> {row['view']}.<br> <b>Condição:</b> {row['condition']}.<br> <b>Nota:</b> {row['grade']}.<br> <b>Metragem sem o porão:</b> {row['sqft_above']}.<br> <b>Metragem do porão:</b> {row['sqft_basement']}.<br> <b>Ano da construção:</b> {row['yr_built']}.<br> <b>Ano da renovação:</b> {row['yr_renovated']}.<br> <b>Zipcode:</b> {row['zipcode']}.<br> <b>Metragem da casa 2015:</b> {row['sqft_living15']}.<br> <b>Metragem do lote 2015:</b> {row['sqft_lot15']}.<br> <b>Novo preço:</b> {row['new_price']}</span><i></div>", max_width=300)
    folium.Marker(location=[row['lat'], row['long']],
                  icon=folium.Icon(icon='home'),
                  popup=popup_text, 
                  parse_html=True, 
                  tooltip=row['zipcode'], max_width=600).add_to(marklus) # adicionando os marcadores no mapa 

array = data[['lat', 'long', 'new_price']].values # foi criado um array para usar no mapa de calor
HeatMap(array).add_to(mapa) # adicionando o mapa de calor com as latitudes e longitudes correspondentes e o peso (intensidade) sendo os valores do new_price

folium.LayerControl().add_to(mapa) # essa função é usada para permitir que o usuário visualize e tenha coltrole sobre as camadas do mapa que ele quer ver

mapa.save('mapaint.html') #salva o mapa como um arquivo .html para abrir no navegador
